# Project2 - Host program

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
from pynq import MMIO
ol = Overlay('phase.bit')

# Download your bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()

print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

In [ ]:
dmaIR = ol.streamPh.dma_I_R # First DMA
dmaQT = ol.streamPh.dma_Q_T # Second DMA
ph_ip = ol.streamPh.phasedetector # Your IP

xlnk = Xlnk() # Contiguous Memory Allocator (CMA)

In [ ]:
length = 1024

# Open input/output files
fI = open('input_i.dat','r')
fQ = open('input_q.dat','r')
fG = open('out_gold.dat', 'r')

# Allocate regular numpy arrays to store input and output
inp_I = np.empty([length,], dtype=np.float32)
inp_Q = np.empty([length,], dtype=np.float32)
golden_R = np.empty([length,], dtype=np.float32)
golden_T = np.empty([length,], dtype=np.float32)

# Store data into arrays
for i in range (0, length):
    golden_R[i], golden_T[i] = [np.float32(x) for x in next(fG).split()]
    inp_I[i] = np.float32(next(fI))
    inp_Q[i] = np.float32(next(fQ))

In [ ]:
plt.plot(golden_R)

In [ ]:
plt.plot(golden_T)

In [ ]:
print("Golden thetas at the R peaks are:\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n".format(golden_T[31],golden_T[63],golden_T[95],golden_T[127],golden_T[159],golden_T[191],golden_T[223],golden_T[255],golden_T[287],golden_T[319],golden_T[351],golden_T[383],golden_T[415],golden_T[447],golden_T[479],golden_T[511]))

## Complete the following block

In [1]:
# Allocate CMA array for DMA
o1 = Overlay('/home/xilinx/jupyter_notebooks/detector/detector.bit')
o1.download()
# Copy regular numpy arrays to CMA arrays
dma1 = o1.detector.axi_dma_0
dma2 = o1.detector.axi_dma_1
# Write length using MMIO (we got the address from Vivado)
detector_ip = MMIO(0x43c00000,0x10000)
xInk = XInk()

size = 1024
#in_buffer1 = xInk.cma_array(shape=(size,), dtype=np.float32)
#in_buffer2 = xInk.cma_array(shape=(size,), dtype=np.float32)
out_bufferT = xInk.cma_array(shape=(size,), dtype=np.float32)
out_bufferR = xInk.cma_array(shape=(size,), dtype=np.float32)

detector_ip.write(0x10,size)
t_start = time.time()
# Begin data transfer from/to DMA
dma1.sendchannel.transfer(inp_I)
dma2.sendchannel.transfer(inp_Q)
dma1.recvchannel.transfer(out_bufferT)
dma2.recvchannel.transfer(out_bufferR)
dma1.sendchannel.wait()
dma2.sendchannel.wait()
dma1.recvchannel.wait()
dma2.recvchannel.wait()
t_stop = time.time()

# Free the CMA buffers
in_buffer1.close()
in_buffer2.close()
out_bufferR.close()
out_bufferT.close()

print(t_after_bitstream - t_before_bitstream, 'seconds to execute on hardware')

NameError: name 'Overlay' is not defined

In [ ]:
plt.plot(out_bufferR)

In [ ]:
plt.plot(out_bufferT)

In [ ]:
print("Output thetas at the R peaks are:\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n {}\n".format(out_bufferT[31],out_bufferT[63],out_bufferT[95],out_bufferT[127],out_bufferT[159],out_bufferT[191],out_bufferT[223],out_bufferT[255],out_bufferT[287],out_bufferT[319],out_bufferT[351],out_bufferT[383],out_bufferT[415],out_bufferT[447],out_bufferT[479],out_bufferT[511]))